In [45]:

import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np




In [31]:


df = pd.read_csv("Preprocessado_df.csv")

df['combined'] = df['title'].fillna('') + " " + df['text'].fillna('') # combinar texto com titulo da noticia
df = df.drop(['title'],axis="columns")
df = df.drop(['text'],axis="columns")
df = df.drop(['Unnamed: 0'],axis="columns")
df



,label,combined
0,1,law enforcement high alert following threat co...
1,1,unbelievable obama attorney general say charlo...
2,0,bobby jindal raised hindu us story christian c...
3,1,satan 2 russia unvelis image terrifying new su...
4,1,time christian group sue amazon splc designati...
...,...,...
71532,0,russian steal research trump hack u democratic...
71533,1,watch giuliani demand democrat apologize trump...
71534,0,migrant refuse leave train refugee camp hungar...
71535,0,trump tussle give unpopular mexican leader muc...


In [50]:


X = df['combined']  
y = df['label']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)


max_words = 50000  
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train) 

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

max_len = 100 
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)



In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=32, input_length=max_len)) 
model.add(LSTM(32)) 
model.add(Dropout(0.5))  
model.add(Dense(12, activation='relu'))  
model.add(Dense(1, activation='sigmoid')) 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

c:\Users\alexa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [59]:
# Treinar o modelo
history = model.fit(X_train_pad, y_train, 
                    epochs=2,
                    batch_size=32, 
                    validation_data=(X_test_pad, y_test))

Epoch 1/2
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 46s 28ms/step - accuracy: 0.8693 - loss: 0.3023 - val_accuracy: 0.9452 - val_loss: 0.1457
Epoch 2/2
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 43s 27ms/step - accuracy: 0.9712 - loss: 0.0860 - val_accuracy: 0.9501 - val_loss: 0.1369


In [60]:
# Avaliar o modelo nos dados de teste
test_loss, test_acc = model.evaluate(X_test_pad, y_test)
print(f"Loss no teste: {test_loss}")
print(f"Acurácia no teste: {test_acc}")

671/671 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9499 - loss: 0.1418
Loss no teste: 0.13691094517707825
Acurácia no teste: 0.9500978589057922


In [61]:
model.save('my_model.keras')